# Programming Assignment 1: Point Inside Shape
## Members: Andy Dao, Mikhail Filippov, Patrick Saxton

In this assignment, we are to solve a computation problem of detecting if a given point is inside a shape, and write the algorithm associated with that solution. We are then to analyze the algorithm through a series of techniques learned in class, include testing, proving correctness, and solving for running time.

### Pseudocode
```plaintext
FUNCTION SegmentCheck(p1, p2, p):
    IF (p.x < MIN(p1.x, p2.x) OR p.x > MAX(p1.x, p2.x) OR
        p.y < MIN(p1.y, p2.y) OR p.y > MAX(p1.y, p2.y)) THEN
        RETURN False
    END IF

    IF p1.x == p2.x THEN
        RETURN (p.x == p1.x)
    END IF

    IF p1.y == p2.y THEN
        RETURN (p.y == p1.y)
    END IF

    IF (p.x - p1.x) * (p2.y - p1.y) == (p.y - p1.y) * (p2.x - p1.x) THEN
        RETURN True
    ELSE
        RETURN False
    END IF
END FUNCTION


FUNCTION CheckIntersection(p1, p2, x, y):
    IF ( (p1.y > y AND p2.y > y) OR (p1.y <= y AND p2.y <= y) ) THEN
        RETURN False
    END IF

    x_intersection = p1.x + (y - p1.y) * (p2.x - p1.x) / (p2.y - p1.y)

    RETURN (x <= x_intersection)
END FUNCTION


FUNCTION PointInPolygon(point, polygon):
    IF (LENGTH(polygon) < 3) THEN
        RAISE ERROR "The list of polygon points must have at least 3 points"
    END IF

    inside = False
    num_vertices = LENGTH(polygon)
    (x, y) = (point.x, point.y)
    p1 = polygon[0]

    FOR i FROM 1 TO num_vertices:
        p2 = polygon[i MOD num_vertices]

        IF PointOnSegment(p1, p2, point) THEN
            RETURN True
        END IF

        IF CheckIntersection(p1, p2, x, y) THEN
            inside = NOT inside
        END IF

        p1 = p2
    END FOR

    RETURN inside
END FUNCTION
```

### Algorithm Implemented:

In [2]:
from typing import List, Tuple

class Point:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

def point_on_segment(p1: Point, p2: Point, p: Point) -> bool:
    if not (min(p1.x, p2.x) <= p.x <= max(p1.x, p2.x) and min(p1.y, p2.y) <= p.y <= max(p1.y, p2.y)):
        return False

    if p1.x == p2.x:
        return p.x == p1.x
    if p1.y == p2.y:
        return p.y == p1.y

    return (p.x - p1.x) * (p2.y - p1.y) == (p.y - p1.y) * (p2.x - p1.x)

def check_intersection(p1: Point, p2: Point, x: float, y: float) -> bool:
    if (p1.y > y) == (p2.y > y):
        return False

    x_intersection = p1.x + (y - p1.y) * (p2.x - p1.x) / (p2.y - p1.y)

    return x <= x_intersection

def point_in_polygon(point: Point, polygon: List[Point]) -> bool:
    if len(polygon) < 3:
        raise ValueError("The list of polygon points must have at least 3 points")

    num_of_vertices = len(polygon)
    x, y = point.x, point.y
    inside = False

    p1 = polygon[0]

    for i in range(1, num_of_vertices + 1):
        p2 = polygon[i % num_of_vertices]

        if point_on_segment(p1, p2, point):
            return True

        if check_intersection(p1, p2, x, y):
            inside = not inside

        p1 = p2

    return inside

### Testing Suite:

In [3]:
import unittest

class TestGeometry(unittest.TestCase):

    def test_point_inside_polygon(self):
        polygon = [Point(0, 0), Point(4, 0), Point(4, 4), Point(0, 4)]
        point = Point(2, 2)
        self.assertTrue(point_in_polygon(point, polygon))

    def test_point_outside_polygon(self):
        polygon = [Point(0, 0), Point(4, 0), Point(4, 4), Point(0, 4)]
        point = Point(5, 5)
        self.assertFalse(point_in_polygon(point, polygon))

    def test_point_on_edge_of_polygon(self):
        polygon = [Point(0, 0), Point(4, 0), Point(4, 4), Point(0, 4)]
        point = Point(4, 2)
        self.assertTrue(point_in_polygon(point, polygon))

    def test_polygon_with_less_than_three_points(self):
        polygon = [Point(0, 0), Point(4, 0)]
        point = Point(2, 2)
        with self.assertRaises(ValueError):
            point_in_polygon(point, polygon)

    def test_point_on_horizontal_segment(self):
        p1 = Point(0, 0)
        p2 = Point(4, 0)
        p = Point(2, 0)
        self.assertTrue(point_on_segment(p1, p2, p))

    def test_point_on_vertical_segment(self):
        p1 = Point(0, 0)
        p2 = Point(0, 4)
        p = Point(0, 2)
        self.assertTrue(point_on_segment(p1, p2, p))

    def test_point_on_diagonal_segment(self):
        p1 = Point(0, 0)
        p2 = Point(4, 4)
        p = Point(2, 2)
        self.assertTrue(point_on_segment(p1, p2, p))

    def test_point_not_on_segment(self):
        p1 = Point(0, 0)
        p2 = Point(4, 4)
        p = Point(3, 2)
        self.assertFalse(point_on_segment(p1, p2, p))

    def test_point_outside_segment_bounds(self):
        p1 = Point(0, 0)
        p2 = Point(4, 4)
        p = Point(5, 5)
        self.assertFalse(point_on_segment(p1, p2, p))

    def test_intersection_within_bounds(self):
        p1 = Point(0, 0)
        p2 = Point(4, 4)
        self.assertTrue(check_intersection(p1, p2, 2, 2))

    def test_intersection_outside_bounds(self):
        p1 = Point(0, 0)
        p2 = Point(4, 4)
        self.assertFalse(check_intersection(p1, p2, 5, 5))

    def test_intersection_on_horizontal_edge(self):
        p1 = Point(0, 0)
        p2 = Point(4, 0)
        self.assertFalse(check_intersection(p1, p2, 2, 0))

    def test_point_inside_concave_polygon(self):
        polygon = [Point(0, 0), Point(4, 0), Point(4, 4), Point(2, 2), Point(0, 4)]
        point = Point(1, 1)
        self.assertTrue(point_in_polygon(point, polygon))

    def test_point_outside_concave_polygon(self):
        polygon = [Point(0, 0), Point(4, 0), Point(4, 4), Point(2, 2), Point(0, 4)]
        point = Point(3, 3)
        self.assertTrue(point_in_polygon(point, polygon))

    def test_point_inside_complex_polygon(self):
        polygon = [Point(2, 6), Point(0, 4), Point(1, 1), Point(3, 0), Point(5, 2), Point(4, 4), Point(2, 3)]
        point = Point(2, 2)
        self.assertTrue(point_in_polygon(point, polygon))

    def test_point_outside_complex_polygon(self):
        polygon = [Point(2, 6), Point(0, 4), Point(1, 1), Point(3, 0), Point(5, 2), Point(4, 4), Point(2, 3)]
        point = Point(5, 5)
        self.assertFalse(point_in_polygon(point, polygon))

    def test_point_on_edge_complex_polygon(self):
        polygon = [Point(2, 6), Point(0, 4), Point(1, 1), Point(3, 0), Point(5, 2), Point(4, 4), Point(2, 3)]
        point = Point(2, 4)
        self.assertTrue(point_in_polygon(point, polygon))

    def test_point_on_vertex_complex_polygon(self):
        polygon = [Point(2, 6), Point(0, 4), Point(1, 1), Point(3, 0), Point(5, 2), Point(4, 4), Point(2, 3)]
        point = Point(2, 6)
        self.assertTrue(point_in_polygon(point, polygon))

    def test_point_near_edge_but_outside(self):
        polygon = [Point(2, 6), Point(0, 4), Point(1, 1), Point(3, 0), Point(5, 2), Point(4, 4), Point(2, 3)]
        point = Point(2, 6.1)
        self.assertFalse(point_in_polygon(point, polygon))

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

...................
----------------------------------------------------------------------
Ran 19 tests in 0.009s

OK


### General Form of Usage:

In [18]:
if __name__ == "__main__":
    test_point = Point(176, 100)

    polygon_points = [
        Point(186, 14),
        Point(186, 44),
        Point(175, 115),
        Point(175, 85)
    ]

    print("Point is inside the polygon" if point_in_polygon(test_point, polygon_points) else "Point is outside the polygon")

Point is inside the polygon


### Asymptotic Worst-Case Analysis
-------------------
Given the main function of the algorithm, `point_in_polygon()`, relies on helper functions, we must first analyze those:
- `point_on_segment()`: This function does not do anything demanding and runs in constant time, given that it is a series of `if` statements with calls to `min()` and `max()`, which also compare two objects in constant time. This gives this helper an O(1).
- `check_intersection()`: As with the above function, this function has even less instructions; it includes simple `if` logic and addition and subtraction statements. This gives this helper an O(1).

With these in mind, we can now move on to `point_in_polygon()`. The function is as follows:

-------------------
```py
def point_in_polygon(point: Point, polygon: List[Point]) -> bool:
    if len(polygon) < 3:
        raise ValueError("The list of polygon points must have at least 3 points")

    num_of_vertices = len(polygon)
    x, y = point.x, point.y
    inside = False

    p1 = polygon[0]

```
-------------------
Up to this point, all operations above this statement have been constant time, so we can reduce them to O(1).

-------------------

```py
    for i in range(1, num_of_vertices + 1):
        p2 = polygon[i % num_of_vertices]

        if point_on_segment(p1, p2, point):
            return True

        if check_intersection(p1, p2, x, y):
            inside = not inside

        p1 = p2

    return inside
```
-------------------
We can see the above `for` loop runs for *n* number of times, where *n* represents the number of vertices on the bounding shape. Because we have stated that both `point_on_segment()` and `check_intersection()` are O(1), we know that there are no more statements within this `for` loop that will raise the time any more.

#### **Therefore, the worst-case of our function is O(n).**

-------------------

### Benchmarking: